In [1]:
def map_concat_columns(row):
    aktivitas = row['b5r13a'] if pd.notnull(row['b5r13a']) else ''
    produk = row['b5r13b'] if pd.notnull(row['b5r13b']) else ''
    usaha = row['b5r13c'] if pd.notnull(row['b5r13c']) else ''
    return f"Aktivitas: {aktivitas}. Produk: {produk}. Usaha: {usaha}."


def split_label(series):
    label = series.iloc[0]  # fix FutureWarning
    label_split = label.split('] ')
    code = label_split[0].removeprefix('[')
    label = label_split[1]
    return code, label

def cleaning_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    
    df = df.dropna(subset=["b5r15a_label"])  # kamu lupa simpan hasil dropna
    df = df.dropna(subset=["b5r15b_label"])  # kamu lupa simpan hasil dropna
    df_new = df[["b5r13a","b5r13b","b5r13c","b5r15a_label","b5r15b_label"]].copy()  # fix SettingWithCopyWarning
    q_df = df_new[["b5r13a","b5r13b","b5r13c"]]
    
    text_description = q_df.apply(map_concat_columns, axis=1)

    kbli_df = df_new[["b5r15a_label"]]
    temp_kbli_code_label = kbli_df.apply(split_label, axis=1)
    kbli_codes = temp_kbli_code_label.apply(lambda x: x[0])
    kbli_labels = temp_kbli_code_label.apply(lambda x: x[1])

    kbji_df = df_new[["b5r15b_label"]]
    temp_kbji_code_label = kbji_df.apply(split_label, axis=1)
    kbji_codes = temp_kbji_code_label.apply(lambda x: x[0])
    kbji_labels = temp_kbji_code_label.apply(lambda x: x[1])

    df_new['text_description'] = text_description
    df_new['kbli_code'] = kbli_codes
    df_new['kbli_label'] = kbli_labels
    df_new['kbji_code'] = kbji_codes
    df_new['kbji_label'] = kbji_labels

    clean_df = df_new[['text_description','kbli_code','kbli_label','kbji_code','kbji_label']]
    return clean_df


In [2]:
import pandas as pd
df_path="../dataset/raw_sak_kalteng.xlsx"

## read excel
sheets=pd.read_excel(df_path,sheet_name=None)

# Filter sheet yang berakhiran '_u' saja
filtered_sheets = {name: df for name, df in sheets.items() if name.endswith('_u')}


# Dictionary untuk simpan hasil
cleaned_data = {}

# Proses semua sheet
for sheet_name, df in filtered_sheets.items():
    print(f"Processing sheet: {sheet_name}")
    try:
        cleaned_df = cleaning_dataset(df)
        cleaned_data[sheet_name] = cleaned_df
    except Exception as e:
        print(f"Error in sheet {sheet_name}: {e}")

Processing sheet: 6201_u
Processing sheet: 6202_u
Processing sheet: 6203_u
Processing sheet: 6204_u
Processing sheet: 6205_u
Processing sheet: 6206_u
Processing sheet: 6207_u
Processing sheet: 6208_u
Processing sheet: 6209_u
Processing sheet: 6210_u
Processing sheet: 6211_u
Processing sheet: 6212_u
Processing sheet: 6213_u
Processing sheet: 6271_u


In [3]:
# Gabungkan semua sheet yang berhasil jadi satu DataFrame
combined_df = pd.concat(cleaned_data.values(), ignore_index=True)

# Simpan ke file baru (opsional)
combined_df.to_csv("../dataset/clean_sak_occupation_kalteng.csv", index=False)
